In [1]:
#this project 

In [2]:
import requests 
import sys
import json as j 
import pandas as pd

In [3]:
###########################
#        Entities         #
###########################

class ResponsePokemonList:
    def __init__(self, name):
        self.name = name

class ResponseEvolutionChainList:
    def __init__(self, url):
        self.url = url

class Pokemon: 
    def __init__(self, id, name, base_experience, height, hp, attack, defense, special_attack, special_defense, speed):
        self.id = id
        self.name = name
        self.base_experience = base_experience
        self.height = height
        self.hp = hp
        self.attack = attack
        self.defense = defense  
        self.special_attack = special_attack
        self.special_defense = special_defense
        self.speed = speed


class TypePokemon: 
    def __init__(self, id, typeName):
        self.id = id
        self.typeName = typeName

class AbilityPokemon: 
    def __init__(self, id, abilityName):
        self.id = id
        self.abilityName = abilityName
    
class GameVersion: 
    def __init__(self, id, versionName):
        self.id = id
        self.versionName = versionName

class MoveDetail: 
    def __init__(self, id, move, learnMethod, gameVersion):
        self.id = id
        self.move = move
        self.learnMethod = learnMethod
        self.gameVersion = gameVersion

class EvolutionChain:
    def __init__(self, id, firstForm, secondForm, thirdForm):
        self.id = id
        self.firstForm = firstForm
        self.secondForm = secondForm
        self.thirdForm = thirdForm

In [4]:
"""
this funcion is responsible to test if the response is valid 
params: 
    - res: API response 
return response serialized as json 
"""
def handleResponse(res):
    if res == None:
        raise Exception("Response is empty!") 
    json_response = j.loads(res.text)   
    return json_response  

In [5]:
"""
this function is responsible to obtain the list of pokemons names avaible in API. The names will be used to make the requests to fetch the 
pokemons attributes. 
return a list of ResponsePokemonList class
"""
def getPokemonNameList():
    list_pokemonResponse = []
    list_pokemonName = []

    API_POKENAMES = 'https://pokeapi.co/api/v2/pokemon/?offset=0&limit='+str(sys.maxsize)
    #API_POKENAMES = 'https://pokeapi.co/api/v2/pokemon/?offset=0&limit=10'

    #request API and validate the response 
    try:
        res = requests.get(API_POKENAMES)
        list_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemExit(err)

    #create the object response to populate the pokemon's names list 
    try:
        for i in list_pokemonResponse['results']:
            name_pokemon = ResponsePokemonList(i['name'])
            list_pokemonName.append(name_pokemon)
    except(Exception) as err:
        print("Error while populate list. Error: ", err)

    return list_pokemonName

In [6]:
def getEvoChainURLList():

    evoChain_URL = []
    evoChainURL_list = []
    API_POKEEVOURL = 'https://pokeapi.co/api/v2/evolution-chain?offset=0&limit='+str(sys.maxsize)

    #request API and validate the response 
    try:
        res = requests.get(API_POKEEVOURL)
        evoURL_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemError(err)

    try:
        for i in evoURL_pokemonResponse['results']:
            evoChain_URL = ResponseEvolutionChainList(i['url'])
            evoChainURL_list.append(evoChain_URL)
    except(Exception) as err:
        print("Error while populate list. Error: ", err)

    return evoChainURL_list


In [7]:
#get the name of pokemons 
pokeName_list = getPokemonNameList()

#lists for create the dataframes 
pokedex = []
types = []
abilities = []
gameVersions = []
moves = []

for pokeName in pokeName_list: 

    API_POKEATTRIBUTES = 'https://pokeapi.co/api/v2/pokemon/'+pokeName.name

    #request API and validate the response 
    try:
        res = requests.get(API_POKEATTRIBUTES)
        attributes_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemError(err)

    pokemon_id = attributes_pokemonResponse['id']

    try:

        #populate the pokemons types list 
        for i in attributes_pokemonResponse['types']:
            types.append(TypePokemon(pokemon_id, i['type']['name']))

        #populate the pokemons ability list
        for i in attributes_pokemonResponse['abilities']:
            abilities.append(AbilityPokemon(pokemon_id, i['ability']['name']))

        #populate the game versions of pokemons list
        for i in attributes_pokemonResponse['game_indices']: 
            gameVersions.append(GameVersion(pokemon_id, i['version']['name']))
        
        #populate the pokemons moves list
        for i in attributes_pokemonResponse['moves']: 
            for x in i['version_group_details']: 
                moves.append(MoveDetail(pokemon_id, i['move']['name'], x['move_learn_method']['name'], x['version_group']['name']))
        
        #populate the pokemons stats
        hp = 0 
        attack = 0 
        defense = 0
        special_attack = 0 
        special_defense = 0
        speed = 0
        for i in attributes_pokemonResponse['stats']:
                if i['stat']['name'] == 'hp':
                    hp = i['base_stat']

                if i['stat']['name'] == 'attack':
                    attack = i['base_stat']
                
                if i['stat']['name'] == 'defense':
                    defense = i['base_stat']
                
                if i['stat']['name'] == 'special-attack':
                    special_attack = i['base_stat']
                
                if i['stat']['name'] == 'special-defense':
                    special_defense = i['base_stat']
                
                if i['stat']['name'] == 'speed':
                    speed = i['base_stat']
                


        #populate the pokemons list
        pokedex.append(Pokemon(
                        attributes_pokemonResponse['id'], 
                        attributes_pokemonResponse['name'], 
                        attributes_pokemonResponse['base_experience'], 
                        attributes_pokemonResponse['height'], 
                        hp, 
                        attack, 
                        defense, 
                        special_attack, 
                        special_defense, 
                        speed
                        )
                    )
                    
    except(Exception) as err:
        print("Err", err)

    # print(pokedex)
    # print(types)
    # print(abilities)
    # print(gameVersions)
    # print(moves)

In [ ]:
evoChainURL_list = getEvoChainURLList()

evoChain_list = []

for evochain in evoChainURL_list:

    #request API and validate the response 
    try:
        res = requests.get(evochain.url)
        evoChain_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemError(err)
    
    evoChain_id = evoChain_pokemonResponse['id']
    firstForm = evoChain_pokemonResponse['chain']['species']['name'] 
    
    try:
        for i in evoChain_pokemonResponse['chain']['evolves_to']:
            secondForm = i['species']['name']
            
            for x in i['evolves_to']:
                thirdForm = x['species']['name']
                evoChain_list.append(EvolutionChain(evoChain_id, firstForm, secondForm, thirdForm))

    except(Exception) as err:
        print("Error while populate list. Error: ", err)

In [ ]:
###########################
#  create the dataframes  #
###########################

pokedex_df = pd.DataFrame([x.__dict__ for x in pokedex])
types_df = pd.DataFrame([x.__dict__ for x in types])
abilities_df = pd.DataFrame([x.__dict__ for x in abilities])
gameVersion_df = pd.DataFrame([x.__dict__ for x in gameVersions])
moves_df = pd.DataFrame([x.__dict__ for x in moves])

evolutionChain_df = pd.DataFrame(x.__dict__ for x in evoChain_list)

In [ ]:
#Higher Pokemon
higherPoke_df = pokedex_df[['name', 'height']].sort_values('height', ascending=False) 

KeyError: "None of [Index(['name', 'height'], dtype='object')] are in the [columns]"